In [1]:
import os

openai_key1 = os.getenv("OPENAI_KEY1")

In [3]:
import datasets
test_df = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/openai/openai-turbo/")

/blue/cai6307/y.khan/conda/envs/tabllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

def to_pandas(item):
  return pd.DataFrame(item['table']["rows"],columns=item['table']["header"])

In [4]:
import openai
import pandas as pd

# Replace with your OpenAI API key
openai.api_key = openai_key1

summaries = []

def generate_summary(table, question, title):
    # Linearize the table
    table_str = table.to_markdown()

    # Construct the prompt
    prompt = f"You are an AI assistant tasked with summarizing key points from tabular data based on a given query. The title of the table is {title}. \n\nTable data: \n\n{table_str}. \n\nQuery: \n\n{question}\n\nProvide a concise summary paragraph that captures the main points from the tabular data relevant to the given query." 


    # Call the OpenAI API
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Extract the output text
    output_text = response.choices[0].text.strip()
    return output_text

In [5]:
import time
k_examples = len(validate)
count = 1;

for item in validate.select(range(k_examples)):
    print(count, "\n")
    count += 1
    question = item['query']
    table = to_pandas(item)
    title = item['table']["title"]
    output = generate_summary(table, question, title)
    
    summaries.append(output)
    time.sleep(30)

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 

33 

34 

35 

36 

37 

38 

39 

40 

41 

42 

43 

44 

45 

46 

47 

48 

49 

50 

51 

52 

53 

54 

55 

56 

57 

58 

59 

60 

61 

62 

63 

64 

65 

66 

67 

68 

69 

70 

71 

72 

73 

74 

75 

76 

77 

78 

79 

80 

81 

82 

83 

84 

85 

86 

87 

88 

89 

90 

91 

92 

93 

94 

95 

96 

97 

98 

99 

100 

101 

102 

103 

104 

105 

106 

107 

108 

109 

110 

111 

112 

113 

114 

115 

116 

117 

118 

119 

120 

121 

122 

123 

124 

125 

126 

127 

128 

129 

130 

131 

132 

133 

134 

135 

136 

137 

138 

139 

140 

141 

142 

143 

144 

145 

146 

147 

148 

149 

150 

151 

152 

153 

154 

155 

156 

157 

158 

159 

160 

161 

162 

163 

164 

165 

166 

167 

168 

169 

170 

171 

172 

173 

174 

175 

176 

177 

178 

179 

180 

181 

182 

183 

184 

185 

In [7]:
validate = validate.add_column("generated_summary", summaries)

In [9]:
validate.save_to_disk("./QTSumm/openai-turbo")

Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 454.19 examples/s]


In [1]:
import datasets
validate = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/openai/openai-turbo")

/blue/cai6307/y.khan/conda/envs/tabllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
example = validate.filter(lambda x: x["example_id"] == "1ae05e92-4ca6-483d-9307-488d28b0b31b")
suma = example['generated_summary']
print(suma)

Filter: 100%|██████████| 200/200 [00:00<00:00, 628.32 examples/s]

['The Utah Jazz All-Time Roster includes several players who attended Byu, including Andy Toolson. Toolson played for the Jazz in 1990-91 and 1995-96 as a guard-forward. Other players on the roster who attended Byu include Ira Terrell, Brooks Thompson, and Bob Thornton. Terrell played forward for the Jazz in 1978-79, Thompson played guard in 1996, and Thornton played forward in 1991-92. All of these players are from the United States and have represented the Jazz at some point in their careers.']


In [3]:
import evaluate
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

pred = validate['generated_summary']
ref = validate['summary']
sc = rouge.compute(predictions=pred, references=ref)
bert_score = bertscore.compute(predictions=pred, references=ref, lang = "en")

2024-04-05 11:46:34.099227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 11:47:02.516760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:/opt/slurm/lib64:
2024-04-05 11:47:02.516807: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-05 11:47:04.116367: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 11:4

In [4]:
import numpy as np
sc['rougeLsum'], np.mean(bert_score['f1'])

(0.33035798695507523, 0.8974467810988426)

In [11]:
import evaluate

rouge = evaluate.load('rouge')
rouge_results = rouge.compute(predictions=validate['generated_summary'], references=validate['summary'])
print(rouge_results)

2024-03-26 23:20:37.133807: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 23:20:39.822527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


{'rouge1': 0.45931039618184233, 'rouge2': 0.22554456142012552, 'rougeL': 0.33049328626744745, 'rougeLsum': 0.33018950514143103}


In [12]:
import statistics

bertscore = evaluate.load("bertscore")
bert_score = bertscore.compute(predictions=validate['generated_summary'], references=validate['summary'], lang = "en")

precision = statistics.mean(bert_score['precision'])
recall = statistics.mean(bert_score['recall'])
f1 = statistics.mean(bert_score['f1'])
print("Precision:", precision)
print("\nRecall:", recall)
print("\nf1:", f1) 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8904176038503647

Recall: 0.9051508378982543

f1: 0.8974467810988426


In [14]:
bleu = evaluate.load("bleu")
bleu_result = bleu.compute(predictions=validate['generated_summary'], references=validate['summary'])
print(bleu_result)

{'bleu': 0.14743665111340337, 'precisions': [0.4043920429146178, 0.18540913200723327, 0.10196616369455876, 0.06180619796484736], 'brevity_penalty': 1.0, 'length_ratio': 1.3318449058569621, 'translation_length': 17896, 'reference_length': 13437}


In [13]:
for summary, generated_summary in zip(validate['summary'], validate['generated_summary']):
    
    print(summary, "\n")
    print(generated_summary, "\n\n")

The Green Bay Packers selected Ken Ruettgers with their first round pick in the 1985 NFL Draft. Ruettgers was a Guard from USC. 

In the first round of the 1985 Green Bay Packers Season, the team selected Ken Ruettgers, a guard from USC with the 7th overall pick. The team also selected other notable players such as Rich Moran, a guard from San Diego State, and Walter Stanley, a wide receiver from Mesa State. Overall, the team focused on selecting players for offensive positions, with the exception of Brian Noble, a linebacker from Arizona State, who was selected in the 5th round. 


From year 2000 to 2010, the age of winners of Miss New Hampshire Teen USA is between 15 and 18 years old. In 2000, Kristen O'Neil from Manchester win at age of 17, and in 2010, Morgan Lucas, also from Manchester, win at age of 18. The smallest age winner during this time is Brittany Dube from Hampstead, who is 15 years old when she win in 2005. Through the years, winners' ages not seem to have a special pat